# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.71,88,100,9.61,TF,1730985591
1,1,yellowknife,62.4560,-114.3525,-5.71,93,75,6.69,CA,1730985592
2,2,crane,31.3974,-102.3501,12.16,56,64,6.45,US,1730985593
3,3,pisco,-13.7000,-76.2167,18.03,88,100,4.63,PE,1730985594
4,4,ak'ordat,15.5486,37.8867,35.46,21,0,2.99,ER,1730985595


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# I'd like to go somewhere with highs between 55 and 75 F (roughly 13-23 C)
filtered_city_data = city_data_df.loc[(city_data_df['Max Temp']<=27) & (city_data_df['Max Temp']>=21)\
     & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:]

#Fixing Namibia country code
filtered_city_data.loc[filtered_city_data['Country'].isna(), 'Country'] = 'NA'

# Drop any rows with null values
np.where(pd.isnull(filtered_city_data))
np.where(pd.isna(filtered_city_data))
    #We have no missing or null values


# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,soro,21.2881,86.6858,23.87,79,0,1.14,IN,1730985505
101,101,at taj,24.2049,23.2857,24.32,28,0,4.41,LY,1730985703
152,152,gazipasa,36.2694,32.3179,24.05,33,0,4.12,TR,1730985572
194,194,kailua-kona,19.6406,-155.9956,23.98,64,0,3.60,US,1730985806
197,197,san patricio,28.0170,-97.5169,23.97,92,0,4.40,US,1730985809
230,230,charters towers,-20.1000,146.2667,21.67,63,0,4.15,AU,1730985846
241,241,bowen,-20.0167,148.2333,24.48,76,0,3.20,AU,1730985859
328,328,dakhla,23.6848,-15.9580,26.01,65,0,2.06,EH,1730985960
366,366,mach,29.8637,67.3298,26.19,15,0,2.11,PK,1730986004
442,442,yeppoon,-23.1333,150.7333,21.85,84,0,4.41,AU,1730986087


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[['City', 'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,soro,IN,21.2881,86.6858,79,
101,at taj,LY,24.2049,23.2857,28,
152,gazipasa,TR,36.2694,32.3179,33,
194,kailua-kona,US,19.6406,-155.9956,64,
197,san patricio,US,28.0170,-97.5169,92,
230,charters towers,AU,-20.1000,146.2667,63,
241,bowen,AU,-20.0167,148.2333,76,
328,dakhla,EH,23.6848,-15.9580,65,
366,mach,PK,29.8637,67.3298,15,
442,yeppoon,AU,-23.1333,150.7333,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
limit = 1
radius = 1000 #hotels within 5km of the coordinates
params = {
    "limit":limit,
    "categories": f"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    #print(name_address.status_code)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        show_df = hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
soro - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
gazipasa - nearest hotel: Güven Otel
kailua-kona - nearest hotel: Kona Seaside Hotel
san patricio - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
bowen - nearest hotel: No hotel found
dakhla - nearest hotel: No hotel found
mach - nearest hotel: No hotel found
yeppoon - nearest hotel: No hotel found
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
holualoa - nearest hotel: Kona Hotel
santiago - nearest hotel: apart arturo prat
garachico - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
32,soro,IN,21.2881,86.6858,79,No hotel found
101,at taj,LY,24.2049,23.2857,28,No hotel found
152,gazipasa,TR,36.2694,32.3179,33,Güven Otel
194,kailua-kona,US,19.6406,-155.9956,64,Kona Seaside Hotel
197,san patricio,US,28.0170,-97.5169,92,No hotel found
230,charters towers,AU,-20.1000,146.2667,63,No hotel found
241,bowen,AU,-20.0167,148.2333,76,No hotel found
328,dakhla,EH,23.6848,-15.9580,65,No hotel found
366,mach,PK,29.8637,67.3298,15,No hotel found
442,yeppoon,AU,-23.1333,150.7333,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)